In [1]:
import torch
import json
import clip
from PIL import Image
import os
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from torchmetrics.functional.multimodal import clip_score
from functools import partial
from pipeline_rf import RectifiedFlowPipeline
import random
import hpsv2
from diffusers import AutoPipelineForText2Image

random.seed(2024)
device = "cuda:0"

/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to get repository contents: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/datasets/zhwang/HPDv2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fb80070f880>, 'Connection to huggingface.co timed out. (connect timeout=None)'))


### load model

In [9]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
clip_model, clip_preprocess = clip.load('ViT-L/14@336px')
clip_model = clip_model.to(device)

In [ ]:
model_id = "/data/model/stable-diffusion-2-1"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
sd_pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
sd_pipe = sd_pipe.to(device)
sd_pipe.set_progress_bar_config(disable=True)

In [ ]:
instaflow_pipe = RectifiedFlowPipeline.from_pretrained("/data/model/instaflow_0_9B_from_sd_1_5", torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False) 
### switch to torch.float32 for higher quality

instaflow_pipe.to(device)  ### if GPU is not available, comment this line
instaflow_pipe.set_progress_bar_config(disable=True)

In [ ]:
sdxl_turbo_pipe = AutoPipelineForText2Image.from_pretrained("/data/model/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
sdxl_turbo_pipe.to(device)
sdxl_turbo_pipe.set_progress_bar_config(disable=True)

In [10]:
lcm_pipe = DiffusionPipeline.from_pretrained("/data/model/LCM_Dreamshaper_v7", safety_checker=None, requires_safety_checker=False)
# To save GPU memory, torch.float16 can be used, but it may compromise image quality.
lcm_pipe.to(torch_device=device, torch_dtype=torch.float32)
lcm_pipe.set_progress_bar_config(disable=True)

Loading pipeline components...: 100%|██████████| 6/6 [00:04<00:00,  1.44it/s]
/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/diffusers/pipelines/pipeline_utils.py:766: FutureWarning: `torch_dtype` is deprecated and will be removed in version 0.27.0. 
  deprecate("torch_dtype", "0.27.0", "")
/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/diffusers/pipelines/pipeline_utils.py:769: FutureWarning: `torch_device` is deprecated and will be removed in version 0.27.0. 
  deprecate("torch_device", "0.27.0", "")


## clip score

In [7]:
def get_clip_score(image, text):
    # Load the pre-trained CLIP model and the image

    # Preprocess the image and tokenize the text
    image_input = clip_preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])
    
    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    
    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_input)
    
    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()
    
    return clip_score

### image with prompt test

In [8]:
# Path to the folder containing your images
folder_path = "/home/liutao/workspace/data/ours_1_coco30k"

total_score = 0
count = 0

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can customize the extension check)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Load the image
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        text = os.path.splitext(filename)[0]
        score = get_clip_score(image, text)
        total_score += score
        count += 1
        if count % 1000 == 0:
            print("current num:", count,f"current AVG CLIP Score: {total_score/count}")

print(f"AVG CLIP Score: {total_score/count}") 


current num: 1000 current AVG CLIP Score: 0.23695098876953125
current num: 2000 current AVG CLIP Score: 0.23639662170410156
current num: 3000 current AVG CLIP Score: 0.23684095255533855
current num: 4000 current AVG CLIP Score: 0.23731815338134765
current num: 5000 current AVG CLIP Score: 0.23748295288085938
current num: 6000 current AVG CLIP Score: 0.23706538899739582
current num: 7000 current AVG CLIP Score: 0.2368526393345424
current num: 8000 current AVG CLIP Score: 0.23662448120117188
current num: 9000 current AVG CLIP Score: 0.23674259440104167
current num: 10000 current AVG CLIP Score: 0.23671253051757812
current num: 11000 current AVG CLIP Score: 0.23672534734552556
current num: 12000 current AVG CLIP Score: 0.2367562764485677
current num: 13000 current AVG CLIP Score: 0.23686055814302884
current num: 14000 current AVG CLIP Score: 0.23690252685546875
current num: 15000 current AVG CLIP Score: 0.23700070393880207
current num: 16000 current AVG CLIP Score: 0.2370119743347168
curr

In [ ]:
# Load the .npz file
# data = np.load('/data/20231212/SwiftBrush_reproduce_final20231227/val2014_captions.npz')
# captions = data['captions'][()]
# print(len(captions),captions[0],captions[1])
# data.close()

### load coco30k_caption

In [12]:
coco_f = open('/data/dataset/coco2014-val/annotations/captions_val2014.json')
coco_annotations = json.load(coco_f)
captions = []
for annotation in coco_annotations['annotations']:
    caption = annotation['caption']
    captions.append(caption)
coco_f.close()
print(len(captions),captions[0],captions[1])

202654 A bicycle replica with a clock as the front wheel. A black Honda motorcycle parked in front of a garage.


In [13]:
captions_30k = random.choices(captions, k=30000)
print(len(captions_30k),captions_30k[0],captions_30k[1])

30000 a little green cart filled with assorted suitcases  A woman in an odd outfit on a bed


### instalflow

In [ ]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = instaflow_pipe(prompt=caption, num_inference_steps=1, guidance_scale=0.0).images[0]
    score = get_clip_score(image, caption)
    try:
        image.save("/home/liutao/workspace/data/instaflow_coco30k/"+caption+".jpg")
    except:
        print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

### sdxl_turbo

In [ ]:
#smallset_test >>> instaflow:0.26 sd_1_step:0.138 sd_25_step:0.22
#instaflow coco30k clip_socre: 0.2580452107747396
#sdxl_turbo_4_step coco30K clip_socre: 0.27137984619140626 
#sdxl_turbo_1_step coco30K clip_socre: 0.2724981628417969 
#lcm coco30k_4_step clip_socre: 
#lcm coco30k_1_step clip_socre: 
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = sdxl_turbo_pipe(prompt=caption, num_inference_steps=4, guidance_scale=0.0).images[0]
    score = get_clip_score(image, caption)
    try:
        image.save("/home/liutao/workspace/data/sdxl_turbo_4_step_coco30K/"+caption+".jpg")
    except:
        print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

In [ ]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = sdxl_turbo_pipe(prompt=caption, num_inference_steps=1, guidance_scale=0.0).images[0]
    score = get_clip_score(image, caption)
    try:
        image.save("/home/liutao/workspace/data/sdxl_turbo_1_step_coco30k/"+caption+".jpg")
    except:
        print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

### lcm

In [ ]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = lcm_pipe(prompt=caption, num_inference_steps=4, guidance_scale=8.0, lcm_origin_steps=50, output_type="pil").images[0]
    score = get_clip_score(image, caption)
    # try:
    #     image.save("/home/liutao/workspace/data/lcm_4_step_coco30K/"+caption+".jpg")
    # except:
    #     print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

In [ ]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = lcm_pipe(prompt=caption, num_inference_steps=1, guidance_scale=8.0, lcm_origin_steps=50, output_type="pil").images[0]
    score = get_clip_score(image, caption)
    # try:
    #     image.save("/home/liutao/workspace/data/lcm_1_step_coco30K/"+caption+".jpg")
    # except:
    #     print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

In [ ]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = lcm_pipe(prompt=caption, height=512, width=512, num_inference_steps=4, guidance_scale=8.0, lcm_origin_steps=50, output_type="pil").images[0]
    score = get_clip_score(image, caption)
    try:
        image.save("/home/liutao/workspace/data/lcm_4_step_512_coco30K/"+caption+".jpg")
    except:
        print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

In [14]:
count = 0
total_score = 0
for case_number, caption in enumerate(captions_30k):
    image = lcm_pipe(prompt=caption, height=512, width=512, num_inference_steps=1, guidance_scale=8.0, lcm_origin_steps=50, output_type="pil").images[0]
    score = get_clip_score(image, caption)
    try:
        image.save("/home/liutao/workspace/data/lcm_1_step_512_coco30K/"+caption+".jpg")
    except:
        print(caption)
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

current num: 1000 current avg clip score: 0.21697222900390625
A compact home bathroom with toilet, pedestal sink, and tub/shower.
current num: 2000 current avg clip score: 0.21796038818359376
Street signs from the corner of 8th ave. and 22 3/4 st.
current num: 3000 current avg clip score: 0.21825511678059895
This tennis pro finds himself resting,  leaning against the advertising graphics/sign.
current num: 4000 current avg clip score: 0.21803173065185547
A lady is trying to go ice skating/skiing. 
current num: 5000 current avg clip score: 0.21849376831054687


## hpsv2 score

In [2]:
from generate import generate_single_image, load_model
vae, tokenizer, text_encoder, unet, scheduler, alphas = load_model("/data/", "/data/20231212/SwiftBrush_reproduce_se_parallel/checkpoints/vsd_global_step26000.pth", device)
# Get benchmark prompts (<style> = all, anime, concept-art, paintings, photo)
all_prompts = hpsv2.benchmark_prompts('all') 
path = "/home/liutao/workspace/data/ours_hpsv2"
# Iterate over the benchmark prompts to generate images
for style, prompts in all_prompts.items():
    for idx, prompt in enumerate(prompts):
        # image = instaflow_pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0] 
        # image = sd_pipe(prompt=prompt, num_inference_steps=25, guidance_scale=0.0).images[0]
        image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt,seed=2024)
        # image = sdxl_turbo_pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
        # TextToImageModel is the model you want to evaluate
        image.save(os.path.join(path, style, f"{idx:05d}.jpg")) 
        # <image_path> is the folder path to store generated images, as the input of hpsv2.evaluate().

[INFO] loading student unet checkpoint


In [3]:
hpsv2.evaluate(path) 

Loading model ...
Loading model successfully!


/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


-----------benchmark score ---------------- 
ours_hpsv2 anime           26.41 	 0.1411
ours_hpsv2 concept-art     26.16 	 0.0911
ours_hpsv2 photo           27.18 	 0.1781
ours_hpsv2 paintings       26.20 	 0.1378


In [4]:
from generate import generate_single_image, load_model
vae, tokenizer, text_encoder, unet, scheduler, alphas = load_model("/data/", "/data/20231212/SwiftBrush_reproduce_se_parallel/checkpoints/vsd_global_step8000.pth", device)
# Get benchmark prompts (<style> = all, anime, concept-art, paintings, photo)
all_prompts = hpsv2.benchmark_prompts('all') 
path = "/home/liutao/workspace/data/ours_hpsv2"
# Iterate over the benchmark prompts to generate images
for style, prompts in all_prompts.items():
    for idx, prompt in enumerate(prompts):
        # image = instaflow_pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0] 
        # image = sd_pipe(prompt=prompt, num_inference_steps=25, guidance_scale=0.0).images[0]
        image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt,seed=2024)
        # image = sdxl_turbo_pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
        # TextToImageModel is the model you want to evaluate
        image.save(os.path.join(path, style, f"{idx:05d}.jpg")) 
        # <image_path> is the folder path to store generated images, as the input of hpsv2.evaluate().
hpsv2.evaluate(path) 

[INFO] loading student unet checkpoint
Loading model ...
Loading model successfully!
-----------benchmark score ---------------- 
ours_hpsv2 anime           26.40 	 0.1460
ours_hpsv2 concept-art     26.06 	 0.0990
ours_hpsv2 photo           27.15 	 0.1647
ours_hpsv2 paintings       26.19 	 0.1230


## fid score